<a href="https://colab.research.google.com/github/vincentbriat/Super-resolution-investigation/blob/main/Discriminator_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Checks if the code is in a colab notebook

try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

Run on Colab notebook

In [2]:
if IN_COLAB:
  !pip install basicsr
  drive.mount('/content/drive/')
  !unzip /content/drive/MyDrive/Datasets/DIV2K_valid_HR.zip
  !unzip /content/drive/MyDrive/Datasets/DIV2K_valid_LR_clean.zip
  !unzip /content/drive/MyDrive/Datasets/DIV2K_train_HR.zip
  !unzip /content/drive/MyDrive/Datasets/DIV2K_train_LR_clean.zip
  FOLDER_LR_TEST = 'DIV2K_valid_LR_clean'
  FOLDER_HR_TEST = 'DIV2K_valid_HR'
  FOLDER_LR_TRAIN = 'DIV2K_train_LR_clean'
  FOLDER_HR_TRAIN = 'DIV2K_train_HR'

  STUDENT_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/student.pth'
  STUDENT_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/student.csv'
  GENERATOR_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/generator.pth'
  GENERATOR_RECORDS_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/Models/generator.csv'

  TEACHER_MODEL_PATH = 'drive/MyDrive/ML/Indiv_Project/Second_Year/KD/ESRGAN_models/RealESRGAN_x4plus.pth'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.9/305.9 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 35.6 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214838 sha256=fcdc3a6bd204a9efa64458735bb0206806b17982dc06cb57c8cc1314a5d2f635
  Stored in directory: /root/.cache/pip/wheels/42/d6/3d/6d8c4b8fabeb93dd442eb255f133e8928c75d903fe20e9472c
Successfully built basicsr
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Success

Run on my Windows desktop

In [ ]:
if not IN_COLAB:
  FOLDER_LR_TEST = 'D:\Downloads\Div2k\DIV2K_valid_LR_clean'
  FOLDER_HR_TEST = 'D:\Downloads\Div2k\DIV2K_valid_HR'
  FOLDER_LR_TRAIN = 'D:\Downloads\Div2k\DIV2K_train_LR_clean'
  FOLDER_HR_TRAIN = 'D:\Downloads\Div2k\DIV2K_train_HR'
  
  STUDENT_MODEL_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\student.pth'
  STUDENT_RECORDS_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\student.csv'
  GENERATOR_MODEL_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\generator.pth'
  GENERATOR_RECORDS_PATH='D:\oldDrive\ML\Indiv_Project\Second_Year\KD\Models\generator.csv'
  
  TEACHER_MODEL_PATH = 'D:\oldDrive\ML\Indiv_Project\Second_Year\KD\ESRGAN_models\RealESRGAN_x4plus.pth'

# Setup

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import cv2
import csv
import math
import numpy as np
import os
import queue
import threading
from basicsr.archs.rrdbnet_arch import RRDBNet
import torchvision
from os import listdir, environ, path
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models import vgg19

# Making sure to use the gpu, if available
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_device(torch.device(0))

In [ ]:
class DIV2KLoader(object):
  def __init__(self, low_res_folder, high_res_folder = None, preprocessing_input = None, preprocessing_output = None, include_targets = True):
    self.low_res_folder = low_res_folder
    self.high_res_folder = high_res_folder
    self.img_names = sorted(listdir(low_res_folder))
    self.include_targets = include_targets
    self.len = len(self.img_names)
    
    if include_targets or high_res_folder is not None:
      self.target_names = sorted(listdir(high_res_folder))

    if preprocessing_input is None:
      self.preprocessing_input = torchvision.transforms.ToTensor()
    else:
      self.preprocessing_input = preprocessing_input

    if preprocessing_output is None:
      self.preprocessing_output = torchvision.transforms.ToTensor()
    else:
      self.preprocessing_output = preprocessing_output
  
  def __getitem__(self, i):
    # Get the right image
    img = Image.open(Path(self.low_res_folder).joinpath(self.img_names[i]))
    if self.include_targets or self.high_res_folder is not None:
      target = Image.open(Path(self.high_res_folder).joinpath(self.target_names[i]))
      return self.preprocessing_input(img), self.preprocessing_output(target)
    else:
      return self.preprocessing_input(img)

  def __len__(self):
    return self.len
  
  def restrict_size(self, size):
    if size < len(self.img_names) and size > 0:
      self.len = size
    else:
      self.len = len(self.img_names)
      print(f"Size must be between 0 and {len(self.img_names)}")

In [ ]:
# 0 is resize
# 1 is crop center
# 2 is crop random (requires finish implementation)

MODE = 1
IM_SIZE = 50

def resize_ratio(img, size=IM_SIZE):
  width, height = img.size
  if width > height:
    width, height = size, int(height * size / width)
  else:
    width, height = int(width * size / height), size
  return width, height

def preprocessing_output(img):
  im_w, im_h = img.size
  if MODE == 1:
    img = img.crop(((im_w - 400)/2, (im_h - 400)/2, (im_w + 400)/2, (im_h + 400)/2))
  elif MODE == 2:
    width, height = resize_ratio(img)
    origin_x, origin_y = 0, 0
    img = img.crop((origin_x, origin_y, origin_x + width*4, origin_y + height*4))
  return torchvision.transforms.ToTensor()(img).to(device)

def preprocessing_input(img):
  im_w, im_h = img.size
  if MODE == 1:
    img = img.crop(((im_w - 400)/2, (im_h - 400)/2, (im_w + 400)/2, (im_h + 400)/2)).resize((100, 100))
  elif MODE == 2:
    width, height = resize_ratio(img)
    origin_x, origin_y = 0, 0
    img = img.crop((origin_x, origin_y, origin_x + width*4, origin_y + height*4)).resize((width, height))
  return torchvision.transforms.ToTensor()(img).to(device)

data_test = DIV2KLoader(FOLDER_HR_TEST, FOLDER_HR_TEST, preprocessing_input=preprocessing_input, preprocessing_output=preprocessing_output)
data_test.restrict_size(10)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=1, shuffle=False)

data_train = DIV2KLoader(FOLDER_LR_TRAIN_CLEAN, FOLDER_LR_TRAIN_CLEAN, preprocessing_input=preprocessing_input, preprocessing_output=preprocessing_output)
data_test.restrict_size(10)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=1, shuffle=False)

# Discriminator

In [ ]:
# http://ethereon.github.io/netscope/#/gist/db945b393d40bfa26006

class ConvBlock(nn.Module):
  def __init__(self, in_channels: int, out_channels: int, kernel_size: int, use_act: bool = True, **kwargs):
    super().__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs)
    self.bn = nn.BatchNorm2d(out_channels)
    self.activation = nn.ReLU(inplace = True) if use_act else nn.Identity()

  def forward(self, x):
    return self.activation(self.conv(x))

class ResidualBlock_v1(nn.Module):
  def __init__(self, in_channels):
    super().__init__()
    self.block = nn.Sequential(
        ConvBlock(in_channels, in_channels, 3, stride=1, padding=1),
        ConvBlock(in_channels, in_channels, 3, stride=1, padding=1))

  def forward(self, x):
    return x + self.block(x)

class ResidualBlock_v2(nn.Module):
  def __init__(self, in_channels, mid_channels, **kwargs):
    self.block = nn.Sequential(ConvBlock(in_channels, mid_channels, 1, padding=0, **kwargs),
                               ConvBlock(mid_channels, mid_channels, 3, padding=1, stride=1),
                               ConvBlock(mid_channels, mid_channels*4, 1, padding=0, stride=1, use_act = False))
    self.residual = nn.Identity() if in_channels*4 == mid_channels*4 else ConvBlock(in_channels, in_channels*4, 1, padding=0, stride=1, use_act = False)

  def forward(self, x):
    return self.block(x) + self.residual(x)

class ResNet(nn.Module):
  def __init__(self):
    self.head = nn.Sequential(ConvBlock(3, 64, 7, stride=2), nn.MaxPool2d(3, 2))
    self.body = nn.Identity()
    self.tail = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                              nn.Flatten(),
                              nn.Linear(1))
    
    def forward(self, x):
      return self.tail(self.body(self.head(x)))

class ResNet_18(ResNet):
  def __init__(self):
    architecture = [64, 128, 256, 512]
    self.body = nn.Sequential(*[self._makeBlock(channels) for channels in architecture)]

  def _makeBlock(self, in_channels):
    return nn.Sequential(ResidualBlock_v1(in_channels), ResidualBlock_v1(in_channels))

class Resnet_50(ResNet):
  def __init__(self):
    architecture = [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3
    self.body = self._makeBlock(architecture)
  
  def _makeBlock(self, architecture):
    layers = [ResidualBlock_v2(64, 64, stride=1)]

    for i in range(1, len(self.architecture)):
      in_channels = mid_channels*4
      mid_channels = self.architecture[i]
      layers.append(ResidualBlock_v2(in_channels, mid_channels, stride= 2 if self.architecture[i-1] != self.architecture[i] else 1))

    return nn.Sequential(*layers)

# Follows the guidelines of https://github.com/soumith/ganhacks
class Discriminator(nn.Module):
  def __init__(self, model: nn.Module):
    self.model = model
    self.loss_fn = nn.BCELOSS()
    self.optimizer = torch.optim.Adam(self.parameters())

  def run_training(self, dataloader):
    cumu_loss = 0
    accuracy = 0
    # This dataloader outputs (Generator(LR_images), HR_images)
    for batch, (X, y) in enumerate(dataloader):
      fake, real = X.to(DEVICE), y.to(DEVICE)
      out_real, out_fake = torch.sigmoid(self.model(y.to(DEVICE))), torch.sigmoid(self.model(X.to(DEVICE)))
      
      out = torch.cat([fake, real], dim=1)
      out = torch.sigmoid(self.model(out))
      target = torch.cat([torch.zeros_like(fake), torch.ones_like(real)], dim=1)

      loss = self.loss_fn(out, target)
      cumu_loss += loss.item()
      
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      
      accuracy_tmp = torch.count_nonzero(torch.equal(out > .5, target)) / target.shape[0]
      accuracy += accuracy_tmp

      if batch % 20 == 0:
        print(f'Trained batch {batch} with loss {loss.item()}')
        print(f'Accuracy: {accuracy_tmp}')

    return {'loss': cumu_loss/len(dataloader), 'accuracy': accuracy/len(dataloader)}

class VGGLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.vgg = vgg19(pretrained=True).features[:35].eval().to(DEVICE)

    for param in self.vgg.parameters():
        param.requires_grad = False

    self.loss = nn.MSELoss()

  def forward(self, input, target):
    vgg_input_features = self.vgg(input)
    vgg_target_features = self.vgg(target)
    return self.loss(vgg_input_features, vgg_target_features)

class GAN:
  def __init__(self, gene, disc):
    self.disc = disc
    self.gen = gene
    self.opt_d = torch.optim.Adam(self.disc.parameters(), lr=2e-5)
    self.opt_g = torch.optim.Adam(self.gen.parameters(), lr=2e-5)
    self.lmb = 5e-3
    self.nu = 1e-2
    self.vgg_loss = VGGLoss()
    self.l1_loss = nn.L1Loss()
    
  def run_training(self, dataloader):
    cum_loss_d, cum_loss_g = 0, 0
    accuracy = 0
    for batch, (X, y) in enumerate(dataloader):
      # Train discriminator
      fake = self.gen(X.to(DEVICE))
      y_cuda = y.to(DEVICE)

      loss_g = self.train_one_model(y_cuda, fake, True)
      loss_d = self.train_one_model(y_cuda, fake, False)

      cumu_loss_d += loss_d
      cumu_loss_g += loss_g

      if batch % 20 == 0:
        print(f'Training batch {batch} with discriminator loss {loss_d:.2f} and generator loss {loss_g:.2f}')
      
      out_fake, out_real = torch.sigmoid(fake), torch.sigmoid(y_cuda)
      accuracy_tmp = torch.count_nonzero(out_fake < 0.5) / out_fake.shape[0] + torch.count_nonzero(out_real > 0.5) / out_real.shape[0]
      accuracy += accuracy_tmp
      print(f'Accuracy: {accuracy_tmp}')

    return {'discriminator': cumu_loss_d / len(dataloader),'generator': cumu_loss_g / len(dataloader), 'accuracy': accuracy / len(dataloader)}

  def train_one_model(self, y_cuda, fake, is_generator: bool):
    d_ra_real, d_ra_fake = self.out_ra(self.disc(y_cuda), self.disc(fake if is_generator else fake.detach()))
    loss = self.loss_g(d_ra_real, d_ra_fake, fake, y_cuda) if is_generator else self.loss_d(d_ra_real, d_ra_fake)

    optimizer = self.opt_g if is_generator else self.opt_d
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

  def out_ra(self, out_real, out_fake):
    return torch.sigmoid(out_real - torch.mean(out_fake)),
            torch.sigmoid(out_fake - torch.mean(out_real))

  def loss_d(self, d_ra_real, d_ra_fake):
    return -torch.mean(torch.log(d_ra_real))) - torch.mean(torch.log(1 - d_ra_fake))

  def loss_g(self, d_ra_real, d_ra_fake, fake, real):
    loss_percep = self.vgg_loss(fake, real)
    loss_ra = self.loss_d(d_ra_fake, d_ra_real)
    loss_l1 = self.l1_loss(fake, real)

    return loss_percep + self.lmb * loss_ra + self.nu * loss_l1